In [1]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import numpy as np
import datetime
import plotly.express as px
import joblib
import Funciones as fn
from unidecode import unidecode #  Esta función convierte los caracteres a su forma sin diacríticos, lo que incluye eliminar las tildes.

# **Diseño de la Solución**

In [2]:
i=Image.open('diseño_solución.png','r') 
i.show()

# **Exploración**

### **Conectar BD**

In [3]:
###### para ejecutar sql y conectarse a bd #####
conn=sql.connect('BD_insurance2019dataset')
cur=conn.cursor()

In [4]:
### para verificar las tablas que hay disponibles ###
pd.read_sql_query("SELECT * FROM sqlite_master; ",conn)

,type,name,tbl_name,rootpage,sql
0,table,reclamaciones,reclamaciones,2,CREATE TABLE reclamaciones (Reclamacion_Id INT...
1,table,diagnosticos,diagnosticos,3,CREATE TABLE diagnosticos (Diagnostico_Codigo ...
2,table,regional,regional,42,"CREATE TABLE regional (Regional_id INTEGER, Re..."
3,table,genero,genero,43,"CREATE TABLE genero (Sexo_Cd TEXT, Sexo_desc T..."
4,table,sociodemograficas,sociodemograficas,44,CREATE TABLE sociodemograficas (Afiliado_Id IN...
5,table,utilizaciones,utilizaciones,1328,CREATE TABLE utilizaciones (Fecha_Reclamacion ...


**Cargar Tablas**

In [5]:
regional = pd.read_sql("SELECT * FROM regional; ",conn)
genero = pd.read_sql("SELECT * FROM genero; ",conn)
diagnosticos = pd.read_sql("SELECT * FROM diagnosticos; ",conn)

### **Identificar campos de cruce, verificar que estén en mismo formato y duplicados**

#### **Tabla Diagnósticos**

In [6]:
diagnosticos.info() # Se cuenta con 2 campos, el Diagnostico_Codigo  y el Diagnostico_Desc
diagnosticos.duplicated().sum() #La tabla no tiene duplicados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2724 entries, 0 to 2723
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Diagnostico_Codigo  2724 non-null   object
 1   Diagnostico_Desc    2724 non-null   object
dtypes: object(2)
memory usage: 42.7+ KB


0

In [7]:
diagnosticos.head()

,Diagnostico_Codigo,Diagnostico_Desc
0,-1,Sin Información
1,0,DIAGNÓSTICO PENDIENTE
2,9,DIAGNÓSTICO PENDIENTE
3,A010,FIEBRE TIFOIDEA
4,A021,SEPTICEMIA DEBIDA A SALMONELLA


In [8]:
# Eliminar las filas con Diagnostico_Desc = "DIAGNÓSTICO PENDIENTE"
diagnosticos = diagnosticos[diagnosticos["Diagnostico_Desc"] != "DIAGNÓSTICO PENDIENTE"]

In [9]:
diagnosticos

,Diagnostico_Codigo,Diagnostico_Desc
0,-1,Sin Información
3,A010,FIEBRE TIFOIDEA
4,A021,SEPTICEMIA DEBIDA A SALMONELLA
5,A028,OTRAS INFECCIONES ESPECIFICADAS COMO DEBIDAS A...
6,A047,ENTEROCOLITIS DEBIDA A CLOSTRIDIUM DIFFICILE
...,...,...
2719,Z961,PRESENCIA DE LENTES INTRAOCULARES
2720,Z966,PRESENCIA DE IMPLANTE ORTOPÉDICO ARTICULAR
2721,Z975,PRESENCIA DE DISPOSITIVO ANTICONCEPTIVO (INTRA...
2722,Z980,ESTADO DE DERIVACIÓN INTESTINAL O ANASTOMOSIS


In [10]:
diagnosticos['Diagnostico_Desc'].value_counts()

APENDICITIS, NO ESPECIFICADA                                   2
EDEMA PULMONAR                                                 2
SÍNCOPE Y COLAPSO                                              2
CUERPO EXTRAÑO EN EL OÍDO                                      2
MALESTAR Y FATIGA                                              2
                                                              ..
INFARTO AGUDO DEL MIOCARDIO                                    1
INFARTO SUBSECUENTE DEL MIOCARDIO, DE PARTE NO ESPECIFICADA    1
OTRAS FORMAS DE ENFERMEDAD ISQUÉMICA AGUDA DEL CORAZÓN         1
ENFERMEDAD CARDIOVASCULAR ATEROSCLERÓTICA, ASÍ DESCRITA        1
OTROS ESTADOS POSTQUIRÚRGICOS ESPECIFICADOS                    1
Name: Diagnostico_Desc, Length: 2699, dtype: int64

In [11]:
diagnosticos['Diagnostico_Desc'] = diagnosticos['Diagnostico_Desc'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)
#Se elimina tildes
diagnosticos['Diagnostico_Desc'] = diagnosticos['Diagnostico_Desc'].str.upper() #Se cambia todos los datos a mayúscula
diagnosticos.head()

,Diagnostico_Codigo,Diagnostico_Desc
0,-1,SIN INFORMACION
3,A010,FIEBRE TIFOIDEA
4,A021,SEPTICEMIA DEBIDA A SALMONELLA
5,A028,OTRAS INFECCIONES ESPECIFICADAS COMO DEBIDAS A...
6,A047,ENTEROCOLITIS DEBIDA A CLOSTRIDIUM DIFFICILE


#### **Tabla Género**

In [12]:
genero.info() # Se observa que tiene 2 campos, el Sexo cd y la descripción 
genero.duplicated().sum() #No se tienen duplicados 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sexo_Cd    3 non-null      object
 1   Sexo_desc  3 non-null      object
dtypes: object(2)
memory usage: 176.0+ bytes


0

In [13]:
genero.head() # Es una tabla descriptiva de los valores asociados en otras tablas

,Sexo_Cd,Sexo_desc
0,M,Masculino
1,F,Femenino
2,-1,Sin Informacion


In [14]:
genero['Sexo_desc'] = genero['Sexo_desc'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)
#Se elimina tildes
genero['Sexo_desc'] = genero['Sexo_desc'].str.upper() #Se cambia todos los datos a mayúscula
genero.head()

,Sexo_Cd,Sexo_desc
0,M,MASCULINO
1,F,FEMENINO
2,-1,SIN INFORMACION


#### **Tabla Regional**

In [15]:
regional.info() # Se observa que tiene 2 campos, id de la región y descripción de cada región 
regional.duplicated().sum() #No se tienen duplicados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Regional_id    6 non-null      int64 
 1   Regional_Desc  6 non-null      object
dtypes: int64(1), object(1)
memory usage: 224.0+ bytes


0

In [16]:
regional.head(6) # Es una tabla descriptiva de los valores asociados en otras tablas

,Regional_id,Regional_Desc
0,1,REGIONAL ANTIOQUIA
1,2,REGIONAL CENTRO
2,3,REGIONAL NORTE
3,4,REGIONAL OCCIDENTE
4,5,REGIONAL EJE CAFETERO
5,6,Sin Información


In [17]:
regional['Regional_Desc'] = regional['Regional_Desc'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)
#Se elimina tildes
regional['Regional_Desc'] = regional['Regional_Desc'].str.upper() #Se cambia todos los datos a mayúscula
regional.head(6)

,Regional_id,Regional_Desc
0,1,REGIONAL ANTIOQUIA
1,2,REGIONAL CENTRO
2,3,REGIONAL NORTE
3,4,REGIONAL OCCIDENTE
4,5,REGIONAL EJE CAFETERO
5,6,SIN INFORMACION


In [18]:
# Reemplazar los valores en Regional_id
regional["Regional_id"] = regional["Regional_id"].replace([6], -1)

In [19]:

joblib.dump(regional,'regional.pkl')
joblib.dump(diagnosticos,'diagnosticos.pkl')
joblib.dump(genero,'genero.pkl')

['genero.pkl']